In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install opencv-python-headless

In [ ]:
import os
import cv2
import numpy as np

def extract_frames(video_path, frame_count=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // frame_count

    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (64, 64))
            frames.append(frame)
        else:
            break
    cap.release()
    return np.array(frames)

def load_data(data_dir):
    X = []
    y = []
    labels = {'defense': 0, 'cover': 1, 'flick': 2, 'pull': 3}

    for label in labels:
        videos_dir = os.path.join(data_dir, label)
        videos = os.listdir(videos_dir)
        print(videos_dir)
        for video in videos:
            video_path = os.path.join(videos_dir, video)
            frames = extract_frames(video_path)
            if len(frames) == 10:
                X.append(frames)
                y.append(labels[label])

    X = np.array(X)
    y = np.array(y)
    return X, y

In [ ]:
data_dir = '/content/drive/My Drive/NewCroppedShot'
X, y = load_data(data_dir)

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train = X
X_val = X
y_train = y
y_val = y

X_train = X_train / 255.0
X_val = X_val / 255.0

# (batch_size, time_steps, height, width, channels)
X_train = np.reshape(X_train, (X_train.shape[0], 10, 64, 64, 3))
X_val = np.reshape(X_val, (X_val.shape[0], 10, 64, 64, 3))



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout

def create_LRCN_model(input_shape):
    model = Sequential()

    # Layer 1
    model.add(TimeDistributed(Conv2D(filters=16, kernel_size=(3, 3), activation='tanh', padding='same'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.2)))

    # Layer 2
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='tanh', padding='same')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.2)))

    # Layer 3
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='tanh', padding='same')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    # Flatten the output
    model.add(TimeDistributed(Flatten()))

    # Dense Layer
   # model.add(TimeDistributed(Dense(64, activation='relu')))

    # LSTM Layer
    model.add(LSTM(32))

    # Output Layer
    model.add(Dense(4, activation='softmax'))  # Assuming 4 classes for the output

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
input_shape = X_train.shape[1:]
print(input_shape)
model = create_LRCN_model(input_shape)

#model_path = '/content/drive/My Drive/ShotDetection/LRCN_model.h5'

# Load the model
#model = tf.keras.models.load_model(model_path)

model.summary()

(10, 64, 64, 3)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 10, 64, 64, 16)    448       
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 10, 16, 16, 16)    0         
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 10, 16, 16, 16)    0         
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 10, 16, 16, 32)    4640      
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 10, 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/50
47/47 [==============================] - 26s 482ms/step - loss: 1.3939 - accuracy: 0.2733 - val_loss: 1.3759 - val_accuracy: 0.2476
Epoch 2/50
47/47 [==============================] - 27s 576ms/step - loss: 1.3612 - accuracy: 0.2991 - val_loss: 1.3641 - val_accuracy: 0.2788
Epoch 3/50
47/47 [==============================] - 28s 601ms/step - loss: 1.2515 - accuracy: 0.4357 - val_loss: 1.1770 - val_accuracy: 0.4993
Epoch 4/50
47/47 [==============================] - 26s 559ms/step - loss: 1.1210 - accuracy: 0.4655 - val_loss: 1.0507 - val_accuracy: 0.5115
Epoch 5/50
47/47 [==============================] - 21s 448ms/step - loss: 1.0944 - accuracy: 0.5250 - val_loss: 1.1004 - val_accuracy: 0.5129
Epoch 6/50
47/47 [==============================] - 23s 495ms/step - loss: 1.0217 - accuracy: 0.5589 - val_loss: 0.9467 - val_accuracy: 0.5995
Epoch 7/50
47/47 [==============================] - 21s 452ms/step - loss: 0.9707 - accuracy: 0.5548 - val_loss: 1.1822 - val_accuracy: 0.4885

In [ ]:
import matplotlib.pyplot as plt

# Extract information from the history object
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

# Print history details
print("Epochs:", history.epoch)
print("Params:", history.params)
print("Keys in history.history:", history.history.keys())

# Plot training vs. validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Training accuracy')
plt.plot(epochs, val_acc, 'ro-', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training vs. validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Training loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'history' is not defined

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelBinarizer

# Make predictions
y_pred_prob = model.predict(X)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(y, y_pred, average='macro')
recall = recall_score(y, y_pred, average='macro')
f1 = f1_score(y, y_pred, average='macro')

# Display the metrics
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1 Score: {f1 * 100:.2f}%')

# Detailed classification report
print("\nClassification Report:\n", classification_report(y, y_pred))

24/24 [==============================] - 6s 253ms/step
Precision: 72.91%
Recall: 35.15%
F1 Score: 26.46%

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.05      0.10       192
           1       0.28      0.95      0.44       187
           2       1.00      0.01      0.01       181
           3       0.72      0.40      0.51       179

    accuracy                           0.35       739
   macro avg       0.73      0.35      0.26       739
weighted avg       0.73      0.35      0.26       739



In [ ]:
model.save('/content/drive/My Drive/NewCroppedShot/lrcn-new-data-tuned-model-4label-max.keras')

In [ ]:
import tensorflow as tf

model_path = '/content/drive/My Drive/NewCroppedShot/lrcn-new-data-tuned-model-4label-max.keras'

# Load the model
model = tf.keras.models.load_model(model_path)

In [ ]:
from google.colab import files
import os

uploaded = files.upload()
video_path = ''

# Move the uploaded video to a specific directory
os.makedirs('videos', exist_ok=True)
for file_name in uploaded.keys():
    video_path = 'videos/demo.' + file_name.split('.')[1]
    os.rename(file_name, os.path.join(video_path))

Saving output2.avi to output2 (1).avi


In [ ]:
new_frames = extract_frames(video_path)
new_frames = np.expand_dims(new_frames, axis=0)

prediction = model.predict(new_frames)
predicted_class = np.argmax(prediction, axis=1)
print(prediction)

shot = {0: 'Defensive Shot', 1: 'Cover Shot', 2: 'Flick Shot', 3: 'Pull Shot'}
print(shot[predicted_class[0]])

1/1 [==============================] - 1s 699ms/step
[[6.7014019e-03 9.8848325e-01 3.2190618e-04 4.4934843e-03]]
Cover Shot
